In [1]:
import os
import subprocess

In [2]:
import looker_sdk
from looker_sdk import models

In [3]:
INSTANCE = 'pebl.dev.looker.com'
CONN = 'jon-bq'
PROJECT = 'model_e2e_workflow'
GIT_USER = 'ContrastingSounds'
GIT_TOKEN = os.environ['GITHUB_CLI_TOKEN']

In [4]:
sdk = looker_sdk.init31()

### 1. Create or Validate a database connection

In [5]:
connection = sdk.connection(CONN)

In [6]:
connection.dialect.connection_tests

['connect', 'kill', 'query', 'database_timezone', 'tmp_db', 'cdt']

In [7]:
test_results = sdk.test_connection(connection.name, models.DelimSequence(connection.dialect.connection_tests))
print(test_results)

[DBConnectionTestResult(can={}, connection_string='jdbc:BQDriver::scratchenvironment/empty?withServiceAccount=true&userAgent=Looker%2F7.0.6+%28GPN%3ALooker%3B%29&readTimeout=240000&connectTimeout=240000&useLegacySql=false', message='Can connect', name='connect', status='success'), DBConnectionTestResult(can={}, connection_string=None, message='Can cancel queries', name='kill', status='success'), DBConnectionTestResult(can={}, connection_string=None, message='Can run simple select query', name='query', status='success'), DBConnectionTestResult(can={}, connection_string=None, message='Can find temp schema "looker_scratch"', name='tmp_db', status='success'), DBConnectionTestResult(can={}, connection_string=None, message='Can use persistent derived tables in "looker_scratch"', name='cdt', status='success')]


### 2. Create repo on origin

In [8]:
!rm -rf .git

In [9]:
!git init

Initialized empty Git repository in /Users/looker/Documents/GitHub/model_e2e_workflow/.git/


In [10]:
!git add .

In [11]:
!git commit -m "first commit"

[master (root-commit) 0f7393c] first commit
 2 files changed, 82 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 test.model.lkml


In [12]:
!hub create

Updating origin
https://github.com/ContrastingSounds/model_e2e_workflow


In [13]:
!git push -u origin HEAD

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 678 bytes | 678.00 KiB/s, done.
Total 4 (delta 0), reused 0 (delta 0)
To github.com:ContrastingSounds/model_e2e_workflow.git
 * [new branch]      HEAD -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


In [14]:
# Run `git rev-parse HEAD` to get commit SHA 
# https://stackoverflow.com/questions/4760215/running-shell-command-and-capturing-the-output
git_SHA = subprocess.run(['git', 'rev-parse', 'HEAD'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()
print(git_SHA)

0f7393cbad9ca4829e183725fc7ecca1230f4fd2


### 3. Create Project

In [15]:
sdk.update_session(body=models.WriteApiSession(workspace_id="dev"))

ApiSession(can={'view': True, 'update': True}, workspace_id='dev', sudo_user_id=None)

In [16]:
# CREATE PROJECT
git_remote_url = ''.join(['https://github.com/', GIT_USER, '/', PROJECT, '.git'])
body = models.WriteProject(
    name=PROJECT, 
    git_service_name="github", 
)
project = sdk.create_project(body)

In [17]:
# HTTPS CONFIG – SET USERNAME & PASSWORD / TOKEN
git_remote_url = ''.join(['https://github.com/', GIT_USER, '/', PROJECT, '.git'])
body = models.WriteProject(
    git_username=GIT_USER,
    git_password=GIT_TOKEN,
    git_remote_url=git_remote_url,
)
project = sdk.update_project(PROJECT, body)

In [18]:
# SSH CONFIG – CREATE DEPLOY KEY BEFORE SETTING REMOTE URL
# deploy_key = sdk.create_git_deploy_key(PROJECT)

In [19]:
# SSH CONFIG – SET GIT REMOTE URL
# git_remote_url = ''.join(['git@github.com:', GIT_USER, '/', PROJECT, '.git'])
# body = models.WriteProject(
#     name=project.name,
#     git_remote_url=git_remote_url,
# )
# updated_project = sdk.update_project(PROJECT, body)

In [20]:
# SET PULL REQUEST MODE
body = models.WriteProject(
    name=PROJECT,
    pull_request_mode='required',
)
updated_project = sdk.update_project(PROJECT, body)

### 4. Deploy to Production 

In [21]:
# Setting the local branch to the master commit ref on remote 
body = models.WriteGitBranch(
    ref=git_SHA
)
sdk.update_git_branch(PROJECT, body)

GitBranch(can={}, name='dev-jon-walls-mggt', remote='origin', remote_name='dev-jon-walls-mggt', error=None, message=None, owner_name='Jon Walls', readonly=False, personal=True, is_local=True, is_remote=False, is_production=False, ahead_count=None, behind_count=None, commit_at=1590762402, ref='6d8f1574a35908b2f972a3ea738fef66903b3b7e', remote_ref=None)

In [22]:
sdk.deploy_to_production(PROJECT)

''

In [23]:
### 5. Testing steps

In [24]:
!echo '# Testing deploy hook' >> test.model.lkml

In [25]:
!git add .

In [26]:
!git commit -m 'Testing deploy hook'

[master 0a1ba1b] Testing deploy hook
 1 file changed, 1 insertion(+)


In [27]:
!git push origin

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 366 bytes | 366.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:ContrastingSounds/model_e2e_workflow.git
   0f7393c..0a1ba1b  master -> master


In [28]:
deploy_hook = '/'.join(['', 'webhooks', 'projects', PROJECT, 'deploy'])
print(deploy_hook)

/webhooks/projects/model_e2e_workflow/deploy


In [29]:
import http.client

connection = http.client.HTTPSConnection(INSTANCE)
connection.request("GET", deploy_hook)
response = connection.getresponse()
print("Status: {} and reason: {}".format(response.status, response.reason))

connection.close()

Status: 200 and reason: OK
